In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [4]:
propiedades = pd.read_csv('/home/agustin/Escritorio/escritorio/fiuba/Organizacion de datos/datos para el tp2/set_datos_propiedades.csv')

In [5]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull()),\
                             ['place_name_encoded', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [4]:
propiedades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132495 entries, 0 to 1413024
Data columns (total 11 columns):
place_name_encoded       1132495 non-null int64
property_type_encoded    1132495 non-null int64
price_aprox_usd          1132495 non-null float64
superficie               1132495 non-null float64
Year                     1132495 non-null int64
Month                    1132495 non-null int64
seguridad                1132495 non-null bool
aire                     1132495 non-null bool
gimnasio                 1132495 non-null bool
cochera                  1132495 non-null bool
pileta                   1132495 non-null bool
dtypes: bool(5), float64(2), int64(4)
memory usage: 65.9 MB


# Decision Tree

In [6]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [7]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [9]:
tree = ExtraTreesRegressor()
tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 91.02 % , error = 12388393674.6


In [12]:
for i in range (10):
    tree = ExtraTreesRegressor(warm_start = True)
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
    precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
    error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
    print("{} - Precision = {:.2f} % , error = {}".format(i, precision, error))

0 - Precision = 90.97 % , error = 12456484013.9
1 - Precision = 91.04 % , error = 12365143887.2
2 - Precision = 91.07 % , error = 12317432207.1
3 - Precision = 91.02 % , error = 12390494796.7
4 - Precision = 91.03 % , error = 12378641786.7
5 - Precision = 91.01 % , error = 12402098557.2
6 - Precision = 91.01 % , error = 12399062890.3
7 - Precision = 91.04 % , error = 12363994452.4
8 - Precision = 91.10 % , error = 12277970916.6
9 - Precision = 91.00 % , error = 12409765742.2


#### Warm start no influye mucho por lo que decidimos no utilizarlo.

## Ahora que tenemos una intuicion, probamos cambiando los parametros

In [7]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [8]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

res = []

In [9]:
lista_criterio = ['mse','mae']
lista_max_depth = [None,1,5,10]
lista_features = [0.4,0.6,1.0]
lista_estimators = [5, 10, 20, 50]

for criterio in lista_criterio:
    for max_depth in lista_max_depth:
        for estimator in lista_estimators:
            for feature in lista_features:

                tree = ExtraTreesRegressor(criterion = criterio, max_depth = max_depth, n_estimators = estimator,\
                                max_features = feature)
                tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
                set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply\
                        (lambda x: tree.predict(x)[0],axis = 1)
                precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.loc[:,'price_aprox_usd']) * 100
                error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
                res.append((criterio,max_depth,estimator,feature,precision,error))
                print(criterio,' - ',max_depth,' - ',estimator,' - ',feature)

mse  -  None  -  2  -  1  -  0.4
mse  -  None  -  2  -  1  -  0.6
mse  -  None  -  2  -  1  -  1.0
mse  -  None  -  2  -  10  -  0.4
mse  -  None  -  2  -  10  -  0.6
mse  -  None  -  2  -  10  -  1.0
mse  -  None  -  3  -  1  -  0.4
mse  -  None  -  3  -  1  -  0.6
mse  -  None  -  3  -  1  -  1.0
mse  -  None  -  3  -  10  -  0.4
mse  -  None  -  3  -  10  -  0.6
mse  -  None  -  3  -  10  -  1.0
mse  -  1  -  2  -  1  -  0.4
mse  -  1  -  2  -  1  -  0.6
mse  -  1  -  2  -  1  -  1.0
mse  -  1  -  2  -  10  -  0.4
mse  -  1  -  2  -  10  -  0.6
mse  -  1  -  2  -  10  -  1.0
mse  -  1  -  3  -  1  -  0.4
mse  -  1  -  3  -  1  -  0.6
mse  -  1  -  3  -  1  -  1.0
mse  -  1  -  3  -  10  -  0.4
mse  -  1  -  3  -  10  -  0.6
mse  -  1  -  3  -  10  -  1.0
mse  -  5  -  2  -  1  -  0.4
mse  -  5  -  2  -  1  -  0.6
mse  -  5  -  2  -  1  -  1.0
mse  -  5  -  2  -  10  -  0.4
mse  -  5  -  2  -  10  -  0.6
mse  -  5  -  2  -  10  -  1.0
mse  -  5  -  3  -  1  -  0.4
mse  -  5  -  3  - 

In [11]:
for r in res:
    print("Criterio = {}, max_depth = {}, n_estimators = {}, features = {}, precision = {:.2f} % , error = {}"\
          .format(r[0],r[1],r[2],r[3],r[4],r[5]))

In [11]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[5] < min_error:
        min_error = r[5]
        tupla_min_error = r
    if r[4] > max_precision:
        max_precision = r[4]
        tupla_max_precision = r
        
print("Mayor precision = Criterio = {}, max_depth = {}, n_estimators = {}, features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3],tupla_max_precision[4],tupla_max_precision[5]))
print("Menor error = Criterio = {}, max_depth = {}, n_estimators = {}, features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3],tupla_min_error[4],tupla_min_error[5]))

Mayor precision = Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 1.0, precision = 90.99 % , error = 12436424287.069954
Menor error = Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 1.0, precision = 90.99 % , error = 12436424287.069954


## Tomamos criterio = mse, max_depth = None , n_estimators = 1, features = 1.0

In [56]:
#Probamos distintas combinaciones de columnas, esta es la que dio mejores resultados

columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio','aire','pileta','cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [57]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [58]:
criterio = 'mse'
max_depth = None
estimator = 2
feature = 1.0

In [59]:
tree = ExtraTreesRegressor(criterion = criterio, max_depth = max_depth, n_estimators = estimator,\
                                max_features = feature)
tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)
precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 90.97 % , error = 12458466712.821684


# calculo con los verdaderos datos a analizar

In [60]:
analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")

In [62]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)

In [63]:
resultado = analizar.loc[:,['id','price_usd']]

In [64]:
resultado.to_csv('resultados/Extra_trees.csv', index = False)